# Linear regression assumptions:

1. No Autocorrelation     : Durbin Watson test : ACF Plot
2. No heteroscedasticity  : Breush Test, Goldfeld Test  : Residual vs Fittes
3. No Multicollinearity   : VIF  & Correlation matrix   
4. Normality of Residuals : Jargue -Bera, Shapiro-wilk Test: Normal QQ
5. Linearity of Residuals : scatter plots and Rainbow Test. 

6. target should be numeric and continous
7. theres should be 2 or more independent variables which are numeric/categoric.

# 1.no autocorrelation
(Test needed : Durbin- Watson Test) value b/w 0-4, 0-2 +ve autocorrelation and good for model.
                                             2-4 -ve autocorrelation and bad for model.

In [ ]:
#this is acf plot.

In [ ]:
import statsmodels.tsa.api as smt   ##change lags as per the number of model_name.resid (print separately and see)

acf = smt.graphics.plot_acf(model_name.resid, lags=40 , alpha=0.05)
acf.show()

# 2.normality of residuals. 
                        (test needed: jarque-bera) 
                        The critical chi square value at the 5% level of significance is 5.99. 
                        H0: residuals are normally distributed
                        
                        If the computed value exceeds this value the null hypothesis is rejected.

In [ ]:
from scipy import stats  #gives jarque-bera statistic with its pvalue.
print(stats.jarque_bera(model_name.resid))

In [ ]:
import seaborn as sns

sns.distplot(model_name.resid)

# 3.Linearity of residuals.
                 : there should be linear relation bw each of the idv with dv.
                 we have two options here:
                 opt1:scatter plot the observed values Vs predicted values and scatter plot the Residual Vs predicted values.
                       first plot should be linear and in second, points hsould be distributed around a horizontal line
                  opt2: Rainbow test:
                              

# opt 1:

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

def linearity_test(model, y):
    '''
    Function for visually inspecting the assumption of linearity in a linear regression model.
    It plots observed vs. predicted values and residuals vs. predicted values.
    
    Args:
    * model - fitted OLS model from statsmodels
    * y - observed values
    '''
    fitted_vals = model.predict()
    resids = model.resid

    fig, ax = plt.subplots(1,2)
    
    sns.regplot(x=fitted_vals, y=y, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title('Observed vs. Predicted Values', fontsize=16)
    ax[0].set(xlabel='Predicted', ylabel='Observed')

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title('Residuals vs. Predicted Values', fontsize=16)
    ax[1].set(xlabel='Predicted', ylabel='Residuals')
    
linearity_test(lin_reg, y)    ##give y_train_name['column_name'] here.

# opt 2: rainbow test

In [ ]:
import statsmodels.api as sm
sm.stats.diagnostic.linear_rainbow(res=lin_reg, frac=0.5)  #line_reg is model name

In [ ]:
import scipy.stats as stats
import pylab
from statsmodels.graphics.gofplots import ProbPlot
st_residual = lin_reg.get_influence().resid_studentized_internal     ##line_reg is the model_name
stats.probplot(st_residual, dist="norm", plot = pylab)
plt.show()

EXPECTATION : THE MEAN VALUE OF THE RESIDUALS SHOULD BE ZERO.
    LET's CHECK

In [ ]:
lin_reg.resid.mean()

# 4 -  Homoscedasticity_test(using goldfeld test) OR (Beusch-Wagon Test)


This test is based on the hytpothesis testing where null and alternate hypothesis are:  (ui is the residuals)
$$ H_{0} : \sigma_{u_{i}}~is~constant~across~the~range~of~data $$

$$ H_{a} : \sigma_{u_{i}}~is~not~constant~across~the~range~of~data $$

The residuals should be homoscedastic.

looking at the pvalue , we can reject/accept the null hypothesis

In [ ]:
from statsmodels.compat import lzip
import numpy as np
from statsmodels.compat import lzip
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

model = lin_reg
fitted_vals = model.predict()
resids = model.resid
resids_standardized = model.get_influence().resid_studentized_internal
fig, ax = plt.subplots(1,2)

sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[0], line_kws={'color': 'red'})
ax[0].set_title('Residuals vs Fitted', fontsize=16)
ax[0].set(xlabel='Fitted Values', ylabel='Residuals')
sns.regplot(x=fitted_vals, y=np.sqrt(np.abs(resids_standardized)), lowess=True, ax=ax[1], line_kws={'color': 'red'})
ax[1].set_title('Scale-Location', fontsize=16)
ax[1].set(xlabel='Fitted Values', ylabel='sqrt(abs(Residuals))')

name = ['F statistic', 'p-value']
test = sms.het_goldfeldquandt(model.resid, model.model.exog)
lzip(name, test)

# 5. No multicollinearity
            we check vif values to evaluate multicollinearity and also heatmap
            delete features with vif>4 one by one and build model

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = [variance_inflation_factor(X_constant.values, i) for i in range(X_constant.shape[1])]
pd.DataFrame({'vif': vif[0:]}, index=X.columns).T   ##change 0 into 1 if there is no 'const' column

In [ ]:
##vif will take one idv as target and find raltion with other features. and it will do the same with all features.
whichevr has multicollinearity , r2 will be high and vif score will be high

In [ ]:
sns.heatmapmap(bos.corr(), annot=True)

In [ ]:
sns.pairplot(bos.corr())

# transformation:


In [ ]:
1. 
## for sqrt(X)
df2 = bos.copy()
final_df = df2.transform(lambda x: x**0.5)
final_df.head()

In [ ]:
2.
#for log  ( if data has '0', it will be infinity after log, which is not possible.)
df2 = bos.copy()
df_final1 = df2.transform(lambda X: np.log(X))
df_final1.head()

In [ ]:
3.
# for inverse ( if data has categorical values, 0 will give infinity after transformation, which is not possible)
df_final1 = df2.transform(lambda X: 1/X)
df_final1.head()

In [ ]:
4.
# for exponential  (check for infinity)
df2 = bos.copy()
df_final1 = df2.transform(lambda X: np.exp(X))
df_final1.head()

In [ ]:
5.
only transform target variable.

# feature selection

# 1.using pearson correlation value:

In [ ]:
#Correlation with output variable
cor_target = abs(cor["Price"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
relevant_features    ##check multicollinearity now.

# 2. wrapper method:

# 2.1 backward elimination  ( gives significant column names only as o/p)

In [ ]:
cols = list(x_1_train.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = x_1_train[cols]
    X_1 = sm.add_constant(X_1)                                    ##if constant is added already, remove this.
    model = sm.OLS(y_1_train,X_1).fit()
    p = pd.Series(model.pvalues.values[0:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
pmax=1
cols = list(Xp.columns)
X_1 = Xp.copy()

while (len(cols)>0):
    p = []
    X_1 = X_1[cols]
    model = sm.OLS(ys, X_1).fit()
    p = model.pvalues
    pmax = max(p)
    
    if (pmax>0.05):
        cols = list(p.drop(p[p==pmax].index).index)
        #print(cols)
    else:
        break
        
selected = cols
print(selected)
print(len(selected))

In [ ]:
--------------------------------------------------------

# 2.2 step foraward selection:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Build RF classifier to use in feature selection
clf = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)


# Build step forward feature selection
sfs1 = sfs(clf,k_features = 10,forward=True,
           floating=False, scoring='r2',
           verbose=2,
           cv=5)   #cv is the 

# Perform SFFS
sfs1 = sfs1.fit(X_train, y_train)

In [ ]:
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

In [ ]:
------------------------------------------

# 2.3 RFE  Recursive Feature Elimination

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

model = LinearRegression()

In [ ]:
#Initializing RFE model
rfe = RFE(model, 11)  ##take 11 variables.

In [ ]:
#Transforming data using RFE
X_rfe = rfe.fit_transform(X,y)    ##x1_train, y1_train
#Fitting the data to model
model.fit(X_rfe,y)                #y1_train
print(rfe.support_)
print(rfe.ranking_)

#### another method to try-Recursive feature Elemination and Grid Search Cv

In [ ]:
from sklearn.feature_selection import RFECV
model = LinearRegression()
selector = RFECV(model,cv=5)
selector = selector.fit(X, y)
print(f"Out of {len(X.columns)} features, best number of features {selector.n_features_}")
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.plot(range(1, len(X.columns) + 1), selector.grid_scores_)
plt.show()

In [ ]:
from sklearn.feature_selection import RFECV
model = LinearRegression(fit_intercept=False)
selector = RFECV(model,cv=5)
selector = selector.fit(X, y)
print(f"Out of {len(X.columns)} features, best number of features {selector.n_features_}")

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score")
plt.plot(range(1, len(X.columns) + 1), selector.grid_scores_)
print(X.columns[selector.support_].values)                         ##build another model with these selected features only.
plt.show()

# 3. Embedded Method:

# 3.1 Ridge regression:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import matplotlib
matplotlib.rcParams.update({'font.size': 12})
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
# higher the alpha value, more restriction on the coefficients; 
# low alpha > more generalization, coefficients are barely
rr = Ridge(alpha=0.01) 
# restricted and in this case linear and ridge regression resembles
rr.fit(X_train, y_train)

In [ ]:
rr1000=Ridge(alpha=1000,max_iter=10e5)
rr1000.fit(x_train_base,y_train_base)
ridge_train_score=rr1000.score(x_train_base,y_train_base)
ridge_test_score=rr1000.score(x_test_base,y_test_base)
coeff_used= np.sum(rr1000.coef_!=0)
print(train_score)
print(test_score)
print(coeff_used)

In [ ]:
Ridge_train_score = rr.score(X_train,y_train)
Ridge_test_score = rr.score(X_test, y_test)

In [ ]:
plt.plot(rr.coef_,alpha=0.7,linestyle='none',marker='*',markersize=5,color='red',label=r'Ridge; $\alpha = 0.01$',zorder=7) # zorder for ordering the markers

In [ ]:
plt.plot(rr100.coef_,alpha=0.5,linestyle='none',marker='d',markersize=6,color='blue',label=r'Ridge; $\alpha = 100$') # alpha here is for transparency

#### or use hypertuning by random or grid search

In [ ]:
X=sm.add_constant(x)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
lr=LinearRegression()
ridge=Ridge()
r_params={'alpha':np.arange(0.01,1,0.01)}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs_ridge=RandomizedSearchCV(ridge,param_distributions=r_params)
rs_ridge.fit(x_train,y_train)
rs_ridge.best_params_

In [ ]:
ridge=Ridge(**rs_ridge.best_params_)
ridge.fit(x_train,y_train)

In [ ]:
ridge_train_rsquare=ridge.score(x_train,y_train)   #rsquare value
ridge_test_rsquare=ridge.score(x_test,y_test)
print(ridge_train_rsquare)
print(ridge_test_rsquare)

y_pred_train=ridge.predict(x_train) 
y_pred_test=ridge.predict(x_test)


from sklearn import metrics
rmse_ridge_train=np.sqrt(metrics.mean_squared_error(y_train,y_pred_train))  #for rmse value from train data.
rmse_ridge_test=np.sqrt(metrics.mean_squared_error(y_test,y_pred_test))

print(rmse_ridge_train)
print(rmse_ridge_test)

### ------------------------------------------------------------------------------------------------------------

# 3.2 Lasso regression

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso()
lasso.fit(X_train,y_train)
train_score=lasso.score(X_train,y_train)
test_score=lasso.score(X_test,y_test)
coeff_used = np.sum(lasso.coef_!=0)

In [ ]:
print("number of coeef used", coeffecient_used )
print('r2 score on train data',train_score )   
print('r2 score on test data',test_score)

In [ ]:
lasso001 = Lasso(alpha=0.01, max_iter=10e5)
lasso001.fit(X_train,y_train)

In [ ]:
plt.subplot(1,2,1)
plt.plot(lasso.coef_,alpha=0.7,linestyle='none',marker='*',markersize=5,color='red',label=r'Lasso; $\alpha = 1$',zorder=7) # alpha here is for transparency
plt.plot(lasso001.coef_,alpha=0.5,linestyle='none',marker='d',markersize=6,color='blue',label=r'Lasso; $\alpha = 0.01$') # alpha here is for transparency
plt.xlabel('Coefficient Index',fontsize=16)
plt.ylabel('Coefficient Magnitude',fontsize=16)
plt.legend(fontsize=13,loc=4)

In [ ]:
plt.subplot(1,2,2)
plt.plot(lasso.coef_,alpha=0.7,linestyle='none',marker='*',markersize=5,color='red',label=r'Lasso; $\alpha = 1$',zorder=7) # alpha here is for transparency
plt.plot(lasso001.coef_,alpha=0.5,linestyle='none',marker='d',markersize=6,color='blue',label=r'Lasso; $\alpha = 0.01$') # alpha here is for transparency
plt.plot(lasso00001.coef_,alpha=0.8,linestyle='none',marker='v',markersize=6,color='black',label=r'Lasso; $\alpha = 0.00001$') # alpha here is for transparency
plt.plot(lr.coef_,alpha=0.7,linestyle='none',marker='o',markersize=5,color='green',label='Linear Regression',zorder=2)
plt.xlabel('Coefficient Index',fontsize=16)
plt.ylabel('Coefficient Magnitude',fontsize=16)
plt.legend(fontsize=13,loc=4)
plt.tight_layout()
plt.show()

#### or use hypertuning by randomsearch or gridsearch

In [ ]:
X=sm.add_constant(x)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
lr=LinearRegression()
lasso=Lasso()
l_params={'alpha':np.arange(0.01,1,0.01)}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs_lasso=RandomizedSearchCV(lasso,param_distributions=l_params)
rs_lasso.fit(x_train,y_train)

In [ ]:
print(rs_lasso.best_params_)
lasso=Lasso(**rs_lasso.best_params_)
lasso.fit(x_train,y_train)

In [ ]:
lasso_train_rsquare=lasso.score(x_train,y_train)
print(lasso_train_rsquare)
lasso_test_rsquare=lasso.score(x_test,y_test)
print(lasso_test_rsquare)

y_pred_train=lasso.predict(x_train)
y_pred_test=lasso.predict(x_test)

rmse_lasso_train=np.sqrt(metrics.mean_squared_error(y_train,y_pred_train))
print(rmse_lasso_train)
rmse_lasso_test=np.sqrt(metrics.mean_squared_error(y_test,y_pred_test))
print(rmse_lasso_test)


# 3.3 Elastic Net regression

In [ ]:
# Let's perform a cross-validation to find the best combination of alpha and l1_ratio
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.metrics import r2_score
## if you put l1_ratio as 0 it is lasso, 1 for ridge. there is not use though as we can directly do lasso and ridge
## n_jobs=-1 should be given always as it allows full spec of our system to be used.
cv_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, .995, 1], eps=0.001, n_alphas=100, fit_intercept=True, 
                        normalize=True, precompute='auto', max_iter=2000, tol=0.0001, cv=5, 
                        copy_X=True, verbose=0, n_jobs=-1, positive=False, random_state=None, selection='cyclic')

In [ ]:
cv_model.fit(X_train, y_train)

In [ ]:
print('Optimal alpha: %.8f'%cv_model.alpha_)
print('Optimal l1_ratio: %.3f'%cv_model.l1_ratio_)
print('Number of iterations %d'%cv_model.n_iter_)

In [ ]:
# train model with best parameters from CV
model = ElasticNet(l1_ratio=cv_model.l1_ratio_, alpha = cv_model.alpha_, max_iter=cv_model.n_iter_, fit_intercept=True, normalize = True)
model.fit(X_train, y_train)

In [ ]:
print(r2_score(y_train, model.predict(X_train)))# training data performance
print(r2_score(y_test, model.predict(X_test))) # test data performance

#### or use hypertuning with randomsearch or grid search

In [ ]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
lr=LinearRegression()
en=ElasticNet()

In [ ]:
en_params={'alpha':np.arange(0.01,1,0.01),'l1_ratio':np.arange(0.1,1,0.01)}
from sklearn.model_selection import RandomizedSearchCV
rs_en=RandomizedSearchCV(en,param_distributions=en_params)
rs_en.fit(x_train,y_train)

In [ ]:
rs_en.best_params_

In [ ]:
en=ElasticNet(**rs_en.best_params_)
en.fit(x_train,y_train)


In [ ]:
en_train_rsquare=en.score(x_train,y_train)   #rsquare value
en_test_rsquare=en.score(x_test,y_test)
print(en_train_rsquare)
print(en_test_rsquare)

y_pred_train=en.predict(x_train)
y_pred_test=en.predict(x_test)


from sklearn import metrics
rmse_en_train=np.sqrt(metrics.mean_squared_error(y_train,y_pred_train))  #for rmse value from train data.
rmse_en_test=np.sqrt(metrics.mean_squared_error(y_test,y_pred_test))

print(rmse_en_train)
print(rmse_en_test)


### to make dataframe of all results:

In [ ]:
result=pd.DataFrame({ 'RMSE':[rmse_lasso_train,rmse_lasso_test,rmse_ridge_train,rmse_ridge_test,rmse_en_train,rmse_en_test],
                    'R square value' : [lasso_train_rsquare,lasso_test_rsquare,ridge_train_rsquare,ridge_test_rsquare,
                     en_train_rsquare,en_test_rsquare]},index=['lasso_train','lasso_test','ridge_train','ridge_test','ElasticNet_train','ElasticNet_train'])
    


# general libraries:

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
from scipy.stats import zscore
from scipy.stats import skew

In [ ]:
city=pd.read_csv('C:/Users/SAJAN P MENON/Desktop/week-6/mini project/bigcity.csv',index_col='Unnamed: 0')   ## change the 1st column of dataframe into indexcolumn
city.head()

# for no. and percentage of null values

In [ ]:
columns_with_missing_values =df.columns[df.isnull().any()]       
df[columns_with_missing_values].isnull().sum()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
           # To hold variable names
labels = []
           # To hold the count of missing values for each variable
valuecount = []

# To hold the percentage of missing values for each variable
percentcount = []
					
for col in columns_with_missing_values:
    labels.append(col)
    valuecount.append(df[col].isnull().sum())     ##df is the main dataframe with null values
       # housepricesdata.shape[0] will give the total row count

    percentcount.append(df[col].isnull().sum()/df.shape[0])    ##df is the main dataframe with null values
    ind = np.arange(len(labels))
					
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,18))

					
rects = ax1.barh(ind, np.array(valuecount), color='blue')
ax1.set_yticks(ind)
ax1.set_yticklabels(labels, rotation='horizontal')
ax1.set_xlabel("Count of missing values")
ax1.set_title("Variables with missing values")
				
rects = ax2.barh(ind, np.array(percentcount), color='pink')
ax2.set_yticks(ind)
ax2.set_yticklabels(labels, rotation='horizontal')
ax2.set_xlabel("Percentage of missing values")
ax2.set_title("Variables with missing values")


# this is for analysing whether there are multiple null values in same row at same time.
#should be done on dataset without replacing

In [ ]:
# Lets import seaborn. We will use seaborn to generate our charts
import seaborn as sns

 							
           # We will import matplotlib to resize our plot figure
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(20, 10))

 							
           # cubehelix palette is a part of seaborn that produces a colormap
cmap = sns.cubehelix_palette(light=1, as_cmap=True, reverse=True)
sns.heatmap(df.isnull(), cmap=cmap)


# for selecting only required dtype columns

In [ ]:
auto_int=auto.select_dtypes(include=['float64','int64'])
auto_int.head()

# for plotting box plot for numerical columns:


In [ ]:
col1=aut.columns

for i in range (0,len(col1)):
    if aut[col1[i]].dtypes=='int64' or aut[col1[i]].dtypes=='float64':
        sns.boxplot(y=auto[col1[i]])
        plt.show()

# statistical tests for dropping features :
-we cannot blindly drop features by taking inference from correalation matrix. following tests are used for that:
-all these tests have H0 and H1: for dropping columns, all should prove h0 as false. (that is accept H1)
Parametric Tests:
1)stats.shapiro(df['house_age'])
2)stats.levene(df['house_age'],df['house_price'])  #house_price is the target variable and house_age is IDV and needs to be                                                          dropped.
Non-Parametric Tests:
1)stats.mannwhitneyu(df['transaction_date'],df['house_price'])  #transaction_date is the IDV and needs to be dropped.

# linear regression model - statsmodel/statistics way

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import statsmodels.api as sm
X_constant = sm.add_constant(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_constant,y, test_size = 0.30, random_state = 1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)



lin_reg = sm.OLS(y_train,x_train).fit()
lin_reg.summary()

# linear regression model - sklearn/machine learning way

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
regressor=LinearRegression()
regressor.fit(x1_train,y1_train)
regressor.score(x1_train,y1_train)  #for r square value
regressor.coef_  #for coeffecient value
regressor.intercept_  ##for intercept value

y_pred=regressor.predict(x1_test)
rmse=np.sqrt(metrics.mean_squared_error(y1_test,y_pred))
mae=metrics.mean_absolute_error(y_pred,y_test)
mse=metrics.mean_squared_error(y_pred,y_test)


# for mean absolute percentage error-mape and rmse

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print('MAPE :',mean_absolute_percentage_error(y,model.predict(X)))
print('RMSE :',np.sqrt(mean_squared_error(y, model.predict(X))))

# different graphs/plot avilable:

In [ ]:
1. univariant

    numerical:
        1)distribution plots
        2)kde plot
        3)histogram
        4)boxplot(can be done for multivariant)
    categorical:
        1)bar chart (if you have multiple categories to plot, pass it as a list)
        2)pie chart

2.Bivariant

    numerical vs numerical: 
        1)lineplot
        2)scatter plot
        3)regplot
    categorical vs categorical: (can apply hue)
        1) count plot
        2)stacked bar chart (easy by making cross tab and applying.plot method)(cor1.plot(kind='bar',stacked=True)
        3)grouped bar chart
        4)segmented bar chart
    categorical vs numerical:
        1)boxplot
        2)violin plot
        3)grouped kde
        4)bar chart (on sumamry statistics, valuecount)
        5)strip plot
        6)swarm plot  (superimpose it with boxplots for inference)

In [ ]:
crosstab:

    df1.groupby(['sex','day']).agg({'tip': 'max' })
    pd.crosstab(df1['sex'],df1['day'])

In [ ]:
#for drawing heatmap with mask. upper part wont come.

In [ ]:
def correlation_heat_map(dff):
    corrs = dff.corr()

    # Set the figure size
    fig, ax = plt.subplots(figsize=(20,18))

    # Generate a mask for the upper triangle
    mask = np.zeros_like(corrs, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Plot the heatmap
    ax = sns.heatmap(corrs, mask=mask, annot=True)

    # Resize the labels
    ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=14, rotation=90)
    ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=14, rotation=0)
    plt.show()

In [ ]:
correlation_heat_map(X)

# statistical methods for decission making

### 1.Discrete probability Distribution:

###### 1.a) binomial distribution: (yes/no)
-it is the probability of getting x number of successes in n number of trials.

In [ ]:
f(x)=(n!/(n-x)! * x!) * p power x * q power (n-x)

    import scipy.stats as stats
    stats.binom.pmf(k=3,n=2,p=.868)        # (p=prob of success, n=total no of trails, k= no of success in n trials)
    stats.binom.cdf(k=2,n=3,p=.868)        # the value shown in k will will included in cdf. means, probability of k=2 is also 
                                               included.
    stats.binom.sf(1,3,0.868)              # gives same output as (stats.binom.pmf(3,3,0.868)+stats.binom.pmf(2,3,0.868))
    
    
    mean=n*p (mean is called as expected value)
    variance=n*p*q
    std=np.sqrt(n*p*(1-p))
    
    # for plotting binomial distribution
    n=3
    k=np.arange(0,4)
    p=0.868
    binomial=stats.binom.pmf(k,n,p)
    plt.plot(k,binomial,'o-')
    plt.title('binomial')
    plt.xlabel('number of orders filled correctly')
    plt.ylabel('prob of orders filled correctly')
    plt.show()

###### 1.b). poisson distribution: 
-probability of number of ocurrences over a specified interval of time or other units


f(x)= (mean power x) * (e power (-mean)) / x!

mean= average/mean/expected values of number of occurence
e=2.71828
x=number of success
f(x)=possibility of occurence in an interval

std deviation=np.sqrt(mean)

In [ ]:
    stats.poisson.pmf(x,mean) #x=number of occurence, mean=average occurence.
    stats.poisson.cdf(x,mean)
    
    
    
    stats.poisson.pmf(2,3)  #2 is the number of occurence, 3 is the average occurence /interval.
    stats.poisson.cdf(4,3)  # number of occurence=4. 4 is also included.
    std=np.sqrt(average occurence/interval)
    
    #for plotting poisson distribution
    n=np.arange(0,16)
    rate=3
    poisson=stats.poisson.pmf(n,rate)
    plt.plot(n,poisson, 'o-')
    plt.title('Poisson')
    plt.xlabel('Number of Work related injuries')
    plt.ylabel('Prob of Work related injuries')
    plt.show()

### 2. continous probability distribution:

In [ ]:
    cdf method:
    stats.norm.cdf(z)  ## area under the curve from left is calculated here.
    
    sf methodL
    stats.norm.sf(z) ## area under the curve from right is calculated.
    

In [ ]:
#mean=mode=median in a perfect normal distribution

In [ ]:
#confidence interval:
    1) z_upper=stats.norm.isf(.005)  ## when cofidence interval is 99%. (1%/2 to be taken) #remember .isf calculates from right to left
       z_lower=stats.norm.isf(1-.005)
    2)  x_lower=(z_lower*(std/np.sqrt(n)))+mean
        x_upper=(z_upper*(std/np.sqrt(n)))+mean
        
    3) (z*std.pop/np.sqrt(sample_size))+ pop.mean or sample_mean
        pop.mean or sample_mean-(z*std.pop/np.sqrt(sample_size))
    
    

        or
    method 2: lci,uci=stats.norm.interval( .95,avg,s)  # avg can be mean_pop or sample_pop, s can be sigma or sigma/sqrt(n)

In [1]:
#confidence interval: (when total number of trials and number of success only given)
import  statsmodels.stats.proportion as  SMP 

y=total trials
x=number of success

LCI, UCI    = SMP.proportion_confint(count = x, nobs = y, alpha = z, method = 'normal')

In [ ]:
#How large a sample would be necessary to estimate the population proportion with a margin of error of .05 at 95% confidence? 
#given, 110 people said yes out of 200 people surveyed.

sample_size= np.square((Z@CI)/error_margin)  *  p(true_proportion)  * (1-p)


favour = 110
total = 200
p=favour/total==0.55  (p is true_proportion)
ci = 95
error=.05
z@critical=1.96  (z=stats.norm.isf(.025))


sample size= (np.square(z@critical/error))*0.55*(1-0.55)

# Hypothesis Testing

company says mileage is 25km/hr
h0: mileage is less than 25km/hr

h1: mileage is grater than 25km/hr
    
result: if we get pvalue less than .05, we accept h1. hence it becomes right tail. there are possibility of both one tail/two tail in one sample ttest,two sample paired ttest,two sample independent ttest. as per the h0 declared, it can be right tail or left tail.
But, anova and chi square test are only one tail (right tail) as we check whethr there is any difference . we dont check less/more. hence one tail.

In [ ]:
     test       |    dependent variable |  independent variable

1.  Anova            continous                 categorical
2.  chi_square       categorical               categorical
3.  regression        continous                continous

###### 1.when only proportions are given

Is there evidence to conclude that the number of people travelling from Hyderabad to Mumbai is different from the number of people travelling from Mumbai to Delhi in a week.

Population 1: Hyderabad to Mumbai 
        n1 = 1200
        x1 = 452
        s1 = 12

        
        
Population 2: Mumbai to Delhi 
        n2 = 800
        x2 = 523
        s2 = 18

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

x,p_val=proportions_ztest((452,523),(1200,800))
x,p_val

###### z test (sample size>30)

In [ ]:
normal z test:
z=(mean_sample-mean_population) / (sigma_sample/np.sqrt(total_sample_size))  

###### sample size<30

In [ ]:
import scipy.stats as stats
from scipy.stats             import ttest_1samp,ttest_ind, wilcoxon,ttest_rel
from statsmodels.stats.power import ttest_power




1. one sample ttest:
     t_statistic, p_value = ttest_1samp(daily_intake, 7600)
2. two sample ttest-independent:
    t_statistic, p_value  =  stats.ttest_ind(Weight_Male,Weight_Female)
    print(p_value)  
3. two sample ttest-relative 
    t_statistic, p_value  =  stats.ttest_rel(wt_after, wt_before )  ##after comes first and then before. always
    print(p_value)  

1 One sample ttest
This can be performed on a single continuous variable to check that mean of that population is above or below by how much and what is the interval.
eg-Is mean salary of the population greater than 270000.

2 Two sample ttest
This test can be performed on one categorical column with another continuous column for checking whether the mean of one sub category issame as other sub-category.

example-The mean salary of male is same as mean salary of female or not

3.
2 sample Proportion test
This test can be performed on two categorical column
To check whether the proportion of one subcategory is same as the others.

example- the propotion of male placed is same as proportion of female placed or not.

4 oneway Annova test
This can be done on 1 categorical column with two continous variable

example- The mean of male salary and marks in degree is same as that of female or not 


5 Twoway Annova test
This can be done on 1 categorical with 2 or more continuous variable.

example-The mean of experienced salary and marks in project and degree is same as female or not.

### 4.Annova test: one way  (no need of making dataframe)
-we are comparing different categorical groups in column with target varaible. whether the means(wrt target varaible) of these different groups are same or not
-example: difference in mean of salary between 3 cities. (3 cities are in same columnn). hence,we can say this is an extension of two sample test. there,we only compare between two categories.

import scipy.stats as stats   
    
life_type_df = pd.DataFrame()

df1            = pd.DataFrame({'life_type': 'A', 'life_hours':life_type_A})  ##life_type_A is the sample 1 array
df2            = pd.DataFrame({'life_type': 'B', 'life_hours':life_type_B})  ##life_type_B is the sample 2 array
df3            = pd.DataFrame({'life_type': 'C', 'life_hours':life_type_C})  ##life_type_C is the sample 3 array

life_type_df = life_type_df.append(df1) 
life_type_df = life_type_df.append(df2)
life_type_df = life_type_df.append(df3) 

stats.f_oneway(life_type_A,life_type_B,life_type_C)


### post-hoc analysis:
 to check which group's mean is different:

from statsmodels.stats.multicomp import pairwise_tukeyhsd
print(pairwise_tukeyhsd(life_type_df['dependant_variable'],life_type_df['independent_varaible'],alpha=0.05))

#inference: post-hoc analysis is used to determine which variables has significant difference. anova can only tell whether there is any significant difference or not. upper and lower here will give the range of mean difference of variable A and variable B. as it has 0 in it, we cant reject null hypothesis. hence it is shown as false under "reject"

anova test says we should reject null hypothesis and post-host analysis says we should accept the null hypothesis.

### Anova test: two way (make dataframe)

-we are comparing different categorical groups in column with target varaible. whether the means(wrt target varaible) of these different groups are same or not
-here, we compare multiple categorical columns with one numerical columns. in one way anova, we only take 1 categorical column with one numerical column.
-remember , null nypothesis say population means of a factor are same.if it gets accepted, it means the said factor has an significant effect on dependenet varaible.

In [ ]:
The null hypotheses for each of the sets are given below.

1) The population means of the first factor (Board_SSC) are equal.
2) The population means of the second factor (Course_Degree) are equal.
3) There is no interaction between the two factors -Board_SSC and  Course_Degree
Alternative Hypothesis:

1) The population means of the first factor ((Board_SSC) are not equal.
2) The population means of the second factor (Course_Degree) are not equal.
3) There is an interaction between the two factors - Board_SSC and  Course_Degree

In [ ]:
d0_val  = df1['Brand X'].values
d10_val = df1['Brand Y'].values
d20_val = df1['Brand Z'].values
l_val   = df1['Loc'].values

df1   = pd.DataFrame({'Loc': l_val, 'Brand':'X','Qty': d0_val})
df2   = pd.DataFrame({'Loc': l_val, 'Brand':'Y','Qty': d10_val})
df3   = pd.DataFrame({'Loc': l_val, 'Brand':'Z','Qty': d20_val})

Sale_qty_df = pd.DataFrame()

Sale_qty_df = Sale_qty_df.append(df1) 
Sale_qty_df = Sale_qty_df.append(df2) 
Sale_qty_df = Sale_qty_df.append(df3) 

pd.DataFrame(Sale_qty_df)

In [ ]:
#another type of data arrangement:
table1  = [['Day','Store-A','Store-B','Store-C','Store-D','Store-E'], [1,69, 71, 64, 67, 56],\
           [2, 68, 76, 79, 87, 76], [3, 71, 77, 74, 84, 72], [4, 70, 73, 71, 78, 73], [5, 60, 64, 67, 79, 58]]


headers = table1.pop(0) #

df1 = pd.DataFrame(table1, columns=headers)
print(df1)

df=df1.melt(value_vars=['Store-A','Store-B','Store-C','Store-D','Store-E'],id_vars='Day')
df

In [ ]:
##apply this after making the dataframe.

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model=ols(formula='value~C(variable)+C(Day)+variable:Day',data=df).fit()  ##c for categorical 
aov_table=anova_lm(model,typ=2)

print(aov_table)

# Inference: 

# From the above table: Pvalue for Loc is 0.512 and for Brand is 1.8 * 10^-13, it shows only Brand is a significant variable

In [ ]:
inference:
    1) here the pvalue 0.5518 for the hypothesis 1 is greater than .05 hence we are accepting the null nypothesis 
        and conclude that board_ssc has an effect on salary
    2) here the pvalue 0.8933 for the hypothesis 2 is greater than .05 hence we are accepting the null nypothesis 
        and conclude that Course_Degree has an effect on salary
    3) here the pvalue 0.509 for the hypothesis 2 is greater than .05 hence we are accepting the null nypothesis 
        and conclude that interaction between Board_SSC:Course_Degree have affect on salary

### 5. chi-square test : one-way
- to compare between observed and expected of the same group.that is only one single column in picture.

In [ ]:
#h0: there is no change in expected value and observed value
#h1: there is a change in expected value and obsrved value

In [ ]:
chi_square_stat, p_value = stats.chisquare(observed_values, f_exp=expected_values)
print('At 5 %s level of significance, the p-value is %1.7f' %('%', p_value))
print('At 5 %s level of significance, the chi observed is %1.7f' %('%', chi_square_stat))

### chi-square : two-way
-to compare between two different categorical groups. that is two different columns

taking categorical features from dataframe: (remember:dont take value counts , just pass column nmae. cross tab does value_counts internally)


df_ct=pd.crosstab(df['gender'],df['Churn'])

In [ ]:
chi_sq_Stat, p_value, deg_freedom, exp_freq = stats.chi2_contingency(churn_array)  ## churn_array=[all values of group1]+[all valuesof group2]+[all values of group3]  ##group in the sense row wise
print('Chi-square statistic %3.5f P value %1.6f Degrees of freedom %d' %(chi_sq_Stat, p_value,deg_freedom))

chi_sq_Stat, p_value, deg_freedom, exp_freq = stats.chi2_contingency([[2549,939],[2625,930]])  ##two square brackets are imp

inference:
this example, p value is 0.486579 and > 0.05 so we accept the null hypothesis.
So, we conclude that There is no difference in churn based on gender nature 'male' or 'female'
(we check whether churn is dependent on gender)

In [ ]:
#inference:
# Here since the pvalue is 0.070563 which is greater than 0.05, we will not reject the null hypothesis that there is not 
# significant differenc in quality among the three shifts

In [ ]:
#selecting/picking desired datatype features/columns
df.select_dtypes(include='object').columns
cat_cols

cat_nums=df.select_dtypes(include=['float64','int64']).columns
cat_nums

# dummies

In [ ]:
df1=pd.get_dummies(data=df_main,columns=['gender','region'],drop_first=True)

# EDA

inference from data
boxplots and outlier identification (from describe)
outlier removal
null vaue imputation
dummies value to categorical columns
rename columns if reqd
check the possibility of multicollinearity (heatmap and vif method)

# label encoding

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df1['status']= label_encoder.fit_transform(df1['status']) 
  
df1['status'].unique()

# EDA-easy way

In [ ]:
import pandas_profiling as Profile
Profile.ProfileReport(df)

# for problem with graphs

In [ ]:
matplotlib.use('TkAgg')

# outlier&distribution together

In [ ]:
cols_outlier=df.select_dtypes('object')
cols_outlier=list(cols_outlier)

for i in range (0,len(cols)):
    if (df[cols[i]].dtypes=='int64' or df[cols[i]].dtypes=='float64') & (cols[i] not in cols_outlier):
        print("variable    :    ",cols[i])
        sns.distplot(df[cols[i]],hist=False)
        plt.show()
        sns.boxplot(y=df[cols[i]])
        plt.show()
        print('\n\n\n\n')

#### outlier removal

for i in range (0,len(outlier_list)):
    
        q3=df[outlier_list[i]].quantile(0.75)
        q1=df[outlier_list[i]].quantile(0.25)
        iqr=q3-q1
        df=df[  (df[outlier_list[i]]>=q1-1.5*iqr)  &   (df[outlier_list[i]]<=q3+1.5*iqr)   ]

        
##remember to reset the index after removal, as there is a chance of jumbled up index number which can cause problem while concat.

# checking whether sample is as same as population
-a.	Are both train and test representative of the overall data? How would you ascertain this statistically

In [ ]:
#one sample t test:

#we shall use one sample t test to check the hypothesis:
#fi nd mean and std deviation of the target variable from dataframe
print('Mean is %2.1f Sd is %2.1f' % (df['Churn'].mean(),np.std(df['Churn'],ddof = 1)))

from scipy.stats             import ttest_1samp,ttest_ind, wilcoxon
from statsmodels.stats.power import ttest_power


#first check y_train with above mean
t_statistic, p_value = ttest_1samp(y_train, df['churn'].mean())
print(t_statistic, p_value)

#second, check y_test with above mean
t_statistic, p_value = ttest_1samp(y_test, df['churn'].mean())
print(t_statistic, p_value)

#explain about hypothesis. 
#ho: means of sample and population are same.
#h1: means are different. we need h0 to be accpeted here.

# scaling/transformation

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax=MinMaxScaler()
df['new_column']=minmax.fit_transform(df[['total_bill']])

In [ ]:
from sklearn.preprocessing import StandardScaler
scal=StandardScaler()
X=pd.DataFrame(scal.fit_transform(x),columns=x.columns)  #we dont scale target variable.

# covariance & correlation:

cov(x,y)= summation ( (x-x_mean)(y-y_mean)  ) / (n-1)

cor(x,y)= cov(x,y) / x.std() * y.std()

# selecting categorical and numerical columns/features
-selecting required datatypes only

In [ ]:
num=df.select_dtypes(include=['int64','float64']).columns
cat=df.select_dtypes(include=['object']).columns

print(list(num))   #numerical features
print(list(cat))   #categorical features.

# checking whether sample is as same as population
-a.	Are both train and test representative of the overall data? How would you ascertain this statistically

### Method 1:
#one sample t test:

#we shall use one sample t test to check the hypothesis:
print('Mean is %2.1f Sd is %2.1f' % (df['Churn'].mean(),np.std(df['Churn'],ddof = 1)))

from scipy.stats             import ttest_1samp,ttest_ind, wilcoxon
from statsmodels.stats.power import ttest_power

t_statistic, p_value = ttest_1samp(df['Churn'], 0.1)
print(t_statistic, p_value)

#explain about hypothesis. 
#ho: means of sample and population are same.
#h1: means are different. we need h0 to be accpeted here.

### method 2:
-check describe() of train and test data

# Feature Engineering:

-when number of features are very less, there is a high chance of overfitting. to increase the features use polynomial feature engineering. it can improve the performance.

# subplots:

In [ ]:
f,axs=plt.subplots(2,2,figsize=(8,5) )
sns.boxplot(df['Salary'],ax=axs[0][0])
sns.kdeplot(df['Salary'],ax=axs[1][0])
sns.boxplot(df['lsalary'],ax=axs[0][1])
sns.kdeplot(df['lsalary'],ax=axs[1][1])


plt.show()

# skeleton for modelling/regression model:

In [ ]:
1) Read Data
2) Shape
3)identify categorical and numerical features.
        a) write inference with proportion of categorical data.
3) Target variable imbalance (only in train set. apply model in test(test should be as is)
        a)undersampling majority class
        b)oversampling minority class
        c)SMOTE-synthetic minority oversampling technique
4) Null value imputation:
        a)if missing value less than 10% use median/mean
        b)more than 10%,bfill/ffill. (but it cant be used in cross sectional data where autocorrelation is zero)
        c) more than 70%, make it into dummy variables(1-no miss, 0-miss data)
        d) if target has missing values, drop it.
5) 5 point summary (describe())
        a) atleast write of two points
6) plots/graphs
        a)Important - bivariant analysis (target vs feature)
        b) univariate analysis
        c) correaltion- heatmap
7) outlier detection and removal:
        a) use iqr method ( if outlier are more, mean will be already compromised. hence, a score method wont be effective)
        b)in regression, if target has outlier, dont remove it. instead transform the target variable to include outlier.
8) categorical into dummies
        a)dont convert target into dummies
        b)if range of categorical columns are less(<=5), convert it into categorical(even if it is float/int). but only ater             understanding the data.
        c)if range of categorical are high(>6), keep it as is.
        d)always drop_first=True
9) Train and test split
        a)make data into standardized(z score) when algorithm has distance calcualtion in it.min_max method(normalization) is             only used in artificial neural network.
        b)use 1 sample t test to check whether sample mean of test is same as train. ( sample vs population)
        c)check 5 point summary of both test and train with inference
10) Base model
        a) statistical approach. Logistica regression is preferred mostly though
        b)improve model by vif/feature selection/feature engineering (poly)
11) machine learning techniques:
        a) linear models can use feature selection (vif). ex: logistic and svm
        b)improve model by 
                            a.1)hyperparamter tuning
                            a.2)interaction/feature engineering/PCA
        c)to get full effect of boarder line data, binning can be done and make it new variable.
        d) regularization for overfitting/underfitting.
12) Mapping all results and picking up best algorithm

# R square and Adjusted R square:

adj r square= 1 - (1-r2) *  (n-1)/ (n-k-1)

r2= r square value
n=number of observations
k=no of independent variables-1

when number of features increases, R ssquare will automatically increase irrespective of the significance the added feature has. so, in this case of adding insignificant feature,  k also increases. when k increases (n-k-1) decreases and whole value of (n-1)/ (n-k-1) incrases. hence, adjusted R square value decreases.

when we add significant feature, R square value increases making (1-r2) *  (n-1)/ (n-k-1) value very less. hence, in this circumstance, adjusted r square value increases.

# table for showing results: (after modelling)
    

In [ ]:
result_table=pd.DataFrame({ 'R square value':[train_score_normal,test_score_normal,train_score1,test_score1,regressor_pf_train,regressor_pf_test,rfr_train,rfr_test],
                           'RMSE value':[rmse_train_normal,rmse_test_normal,rmse_train1,rmse_test1,regressor_pf_rmse_train,regressor_pf_rmse_test,rfr_rmse_train,rfr_rmse_test]},
                         index=['train_base model','test_base mode','train_after feature selection','test_after feature selection','train_polynomial','test_polynomial','train_random forrest','test_random forrest'])



In [ ]:
result_table
result_table1=result_table.reset_index()
result_table1

In [ ]:
plt.figure(figsize=(18,4))
sns.barplot(x='index',y='R square value',data=result_table1)
plt.show()

In [ ]:
plt.figure(figsize=(18,4))
sns.barplot(x='index',y='RMSE value',data=result_table1)
plt.show()

# important features/significant features.
-which variables are affecting the target the most and explain the relationship

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(random_state=1)
rfr.fit(xp_df,yp_df)


rfr.fit(X_nc,Y)
cols=X_nc.columns
rfr.feature_importances_
fi=pd.DataFrame(rfr.feature_importances_,index=cols,columns=['importance'])
fi

In [ ]:
#to show full rows and columns:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)